In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.initializers import Constant, HeNormal
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/redes/indicadores-filtrados-bundesliga-5-cambios.csv'
df = pd.read_csv(path, sep=",")
df = df.dropna(subset=["resultado_partido"]).reset_index(drop=True)
print(df["resultado_partido"].unique())

Mounted at /content/drive
['1' 'X' '2']


In [ ]:
# === Codificar variable objetivo ===
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df["resultado_partido"])
y_categorical = to_categorical(y_encoded)


# === Dividir en entrenamiento y test ===
X = df.drop(["resultado_partido", "resultado_local", "resultado_visitante", "jornada", "id_indicadores_equipo_prepartido", "id_partido", "temporada"], axis=1)
from sklearn.preprocessing import MinMaxScaler
X_scaled = MinMaxScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42)


In [ ]:

# === Inicializar resultados ===
resultsDF = pd.DataFrame(columns=["epocas", "batch", "optimizador", "callbacks", "red", "accuracy"])

# === Hiperparámetros ===
opciones_epocas = [50, 80]
opciones_batch = [16, 48]
opciones_optimizador = ["SGD", "adam"]
opciones_callbacks = [0, 1]
opciones_red = [0, 1, 2, 3, 4, 5, 6]


In [ ]:
# === Grid search ===
for epocas in opciones_epocas:
    for batch in opciones_batch:
        for opt in opciones_optimizador:
            for cb in opciones_callbacks:
                for red in opciones_red:

                  if red == 0:
                      model = Sequential([
                          Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
                          Dense(3, activation='softmax')
                      ])

                  elif red == 1:
                      model = Sequential([
                          Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
                          Dense(64, activation='relu'),
                          Dense(32, activation='relu'),
                          Dense(3, activation='softmax')
                      ])

                  elif red == 2:
                      model = Sequential([
                          Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
                          Dense(128, activation='relu'),
                          Dropout(0.4),
                          Dense(64, activation='relu'),
                          Dropout(0.3),
                          Dense(3, activation='softmax')
                      ])

                  elif red == 3:
                      model = Sequential([
                          Dense(128, activation='relu', kernel_regularizer=l2(0.001), input_shape=(X_train.shape[1],)),
                          BatchNormalization(),
                          Dropout(0.3),
                          Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
                          BatchNormalization(),
                          Dense(32, activation='relu'),
                          Dense(3, activation='softmax')
                      ])

                  elif red == 4:
                      model = Sequential([
                          Dense(128, activation='relu', kernel_initializer=HeNormal(), input_shape=(X_train.shape[1],)),
                          BatchNormalization(),
                          Dropout(0.3),
                          Dense(64, activation='relu', kernel_initializer=HeNormal()),
                          BatchNormalization(),
                          Dense(32, activation='relu'),
                          Dropout(0.2),
                          Dense(3, activation='softmax')
                      ])

                  elif red == 5:
                      model = Sequential([
                          Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
                          Dense(64, activation='relu'),
                          Dense(32, activation='relu'),
                          Dense(64, activation='relu'),
                          Dense(3, activation='softmax')
                      ])

                  elif red == 6:
                      model = Sequential([
                          Dense(256, activation='relu', kernel_initializer=HeNormal(), input_shape=(X_train.shape[1],)),
                          BatchNormalization(),
                          Dropout(0.3),
                          Dense(128, activation='relu'),
                          BatchNormalization(),
                          Dropout(0.3),
                          Dense(32, activation='relu'),
                          Dense(3, activation='softmax')
                      ])

                  # Compilación
                  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

                  # Pesos de clase
                  y_train_labels = np.argmax(y_train, axis=1)
                  class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_labels), y=y_train_labels)
                  class_weight_dict = dict(enumerate(class_weights))

                  # Callbacks
                  callbacks_list = []
                  if cb == 1:
                      early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
                      reduce_lr = ReduceLROnPlateau(factor=0.2, patience=5)
                      callbacks_list = [early_stopping, reduce_lr]

                  # Entrenamiento
                  history = model.fit(
                      X_train, y_train,
                      epochs=epocas,
                      batch_size=batch,
                      validation_split=0.2,
                      class_weight=class_weight_dict,
                      callbacks=callbacks_list,
                      verbose=0
                  )

                  # Evaluación
                  loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

                  # Guardar el mejor modelo de cada tipo de red
                  model_path = f"/content/drive/MyDrive/Colab Notebooks/redes/mejor_modelo_red_{red}_resultado_bundesliga-5-cambios.h5"
                  try:
                      # Si ya hay un modelo guardado, lo cargamos para comparar su accuracy
                      from tensorflow.keras.models import load_model
                      modelo_guardado = load_model(model_path)
                      _, acc_guardado = modelo_guardado.evaluate(X_test, y_test, verbose=0)
                      if accuracy > acc_guardado:
                          model.save(model_path)
                          print(f"✅ Modelo actualizado para red {red} con accuracy {accuracy:.4f}")
                  except:
                      # Si no existe, lo guardamos directamente
                      model.save(model_path)
                      print(f"📁 Modelo guardado por bundesliga vez para red {red} con accuracy {accuracy:.4f}")

                  print(f"Red {red} | Opt: {opt} | Ep: {epocas} | Batch: {batch} | CB: {cb} → Acc: {accuracy:.4f}")

                  # Guardar resultados
                  resultsDF.loc[len(resultsDF)] = [epocas, batch, opt, cb, red, accuracy]

# === Exportar resultados ===
resultsDF.to_csv("/content/drive/MyDrive/Colab Notebooks/redes/modelos_resultado_bundesliga-5-cambios.csv", index=False)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


📁 Modelo guardado por bundesliga vez para red 0 con accuracy 0.3534
Red 0 | Opt: SGD | Ep: 50 | Batch: 16 | CB: 0 → Acc: 0.3534


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


📁 Modelo guardado por bundesliga vez para red 1 con accuracy 0.3103
Red 1 | Opt: SGD | Ep: 50 | Batch: 16 | CB: 0 → Acc: 0.3103


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


✅ Modelo actualizado para red 4 con accuracy 0.4483
Red 4 | Opt: adam | Ep: 80 | Batch: 48 | CB: 1 → Acc: 0.4483


Red 5 | Opt: adam | Ep: 80 | Batch: 48 | CB: 1 → Acc: 0.4052


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Red 6 | Opt: adam | Ep: 80 | Batch: 48 | CB: 1 → Acc: 0.4138


In [ ]:


# Paso 1: Evaluar los modelos guardados
mejor_acc = -1
mejor_modelo = None
mejor_red = None

for red in range(7):
    try:
        path = f"/content/drive/MyDrive/Colab Notebooks/redes/mejor_modelo_red_{red}_resultado_bundesliga-5-cambios.h5"
        model = load_model(path)
        loss, acc = model.evaluate(X_test, y_test, verbose=0)
        print(f"Red {red} --> Accuracy = {acc:.4f}")
        if acc > mejor_acc:
            mejor_acc = acc
            mejor_modelo = model
            mejor_red = red
    except Exception as e:
        print(f"Error con modelo red {red}: {e}")

print(f"\n🎯 Mejor modelo: red {mejor_red} con accuracy = {mejor_acc:.4f}")

# Paso 2: Obtener pesos del primer Dense layer
primer_dense = None
for layer in mejor_modelo.layers:
    if "Dense" in layer.__class__.__name__:
        primer_dense = layer
        break

if primer_dense is not None:
    pesos, _ = primer_dense.get_weights()  # pesos.shape = (n_variables, n_neuronas)
    importancia = np.mean(np.abs(pesos), axis=1)  # media de pesos por variable (input)

    # Paso 3: Asociar importancia con nombres de columnas
    nombres_variables = X.columns  # asegúrate de que esté disponible X original sin escalar
    importancia_df = pd.DataFrame({
        "variable": nombres_variables,
        "importancia": importancia
    })

    top_15 = importancia_df.sort_values(by="importancia", ascending=False).head(15)
    print("\n🔝 Top 15 variables más importantes según los pesos del primer layer:\n")
    print(top_15.to_string(index=False))
else:
    print("❌ No se encontró una capa Dense inicial en el mejor modelo.")


Red 0 --> Accuracy = 0.5000


Red 1 --> Accuracy = 0.4397


Red 2 --> Accuracy = 0.4397
Red 3 --> Accuracy = 0.4138


Red 4 --> Accuracy = 0.4483


Red 5 --> Accuracy = 0.4310
Red 6 --> Accuracy = 0.4569

🎯 Mejor modelo: red 0 con accuracy = 0.5000

🔝 Top 15 variables más importantes según los pesos del primer layer:

                                                       variable  importancia
                    proporcion local corners en contra en sitio     0.095123
                  proporcion visitante cambios 45 a 60 en sitio     0.092366
                 proporcion visitante goles marcados en general     0.091868
                           proporcion visitante puntos en sitio     0.090634
                proporcion visitante corners a favor en general     0.090600
                           porcentaje local perdidos en general     0.090540
                    proporcion local corners a favor en general     0.089689
                 proporcion local cambios goleadores en general     0.089286
 proporcion local cambios centrocampistas a defensas en general     0.089047
                     porcentaje local mas 0,5 marcados en 

In [ ]:
# Paso 4: Evaluar peso total de variables que contienen "cambios"
importancia_total = importancia_df["importancia"].sum()

# Filtrar variables que contienen "cambios" (case insensitive)
importancia_cambios = importancia_df[importancia_df["variable"].str.contains("cambios", case=False)]

suma_cambios = importancia_cambios["importancia"].sum()
proporcion = (suma_cambios / importancia_total) * 100

print(f"\n📈 Proporción de importancia atribuida a variables relacionadas con 'cambios': {proporcion:.2f}% del total")



📈 Proporción de importancia atribuida a variables relacionadas con 'cambios': 44.78% del total


In [ ]:
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


# === Arquitectura Wide & Deep ===
input_layer = Input(shape=(X_train.shape[1],))

# Parte wide (lineal directa)
wide = Dense(3, activation='linear')(input_layer)

# Parte deep
deep = Dense(128, activation='relu')(input_layer)
deep = BatchNormalization()(deep)
deep = Dropout(0.3)(deep)
deep = Dense(64, activation='relu')(deep)
deep = Dropout(0.2)(deep)
deep = Dense(32, activation='relu')(deep)
deep = Dense(3, activation='linear')(deep)  # preactivación softmax

# Combinar wide + deep
combined = Concatenate()([wide, deep])
output = Dense(3, activation='softmax')(combined)

# Modelo final
model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Pesos de clase
y_labels = np.argmax(y_train, axis=1)
cl_weights = compute_class_weight('balanced', classes=np.unique(y_labels), y=y_labels)
class_weight_dict = dict(enumerate(cl_weights))

# Entrenar
model.fit(X_train, y_train,
          validation_split=0.2,
          epochs=80,
          batch_size=32,
          class_weight=class_weight_dict,
          callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
          verbose=0)

# Evaluar
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"✅ Accuracy Wide & Deep: {acc:.4f}")


✅ Accuracy Wide & Deep: 0.3621


In [ ]:
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# === Bloques de MLP Mixer ===

def mixer_block(x, units, dropout_rate=0.0):
    # Normalización
    shortcut = x
    x = LayerNormalization()(x)

    # MLP: primera capa
    x = Dense(units, activation='relu')(x)
    x = Dropout(dropout_rate)(x)

    # MLP: segunda capa
    x = Dense(shortcut.shape[-1], activation=None)(x)
    x = Add()([x, shortcut])  # skip connection

    return x

# === Modelo completo ===

def build_mlp_mixer(input_dim, num_classes=3, depth=4, hidden_dim=128, dropout_rate=0.1):
    inputs = Input(shape=(input_dim,))
    x = Dense(hidden_dim)(inputs)

    # Apilar bloques Mixer
    for _ in range(depth):
        x = mixer_block(x, hidden_dim * 2, dropout_rate)

    x = LayerNormalization()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

# === Entrenamiento ===

model = build_mlp_mixer(input_dim=X_train.shape[1], num_classes=3, depth=4, hidden_dim=128, dropout_rate=0.2)
model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train,
          validation_split=0.2,
          epochs=80,
          batch_size=32,
          class_weight=class_weight_dict,
          callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
          verbose=0)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"✅ Accuracy MLP-Mixer tabular: {acc:.4f}")


✅ Accuracy MLP-Mixer tabular: 0.4310


In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

# === Paso 1: Autoencoder ===

input_dim = X_train.shape[1]
encoding_dim = 32  # Puedes ajustar este número

# Encoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
encoded_output = Dense(encoding_dim, activation='relu')(encoded)

# Decoder (no se usará después, solo para reconstrucción)
decoded = Dense(64, activation='relu')(encoded_output)
decoded = Dense(128, activation='relu')(decoded)
decoded_output = Dense(input_dim, activation='sigmoid')(decoded)

# Modelo autoencoder
autoencoder = Model(inputs=input_layer, outputs=decoded_output)
autoencoder.compile(optimizer='adam', loss='mse')

# Entrenar autoencoder
autoencoder.fit(X_train, X_train,
                epochs=100,
                batch_size=32,
                validation_split=0.2,
                callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
                verbose=0)

# === Paso 2: Usar encoder para representar los datos ===

encoder = Model(inputs=input_layer, outputs=encoded_output)

X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)

# === Paso 3: Clasificador sobre las representaciones ===

from tensorflow.keras.models import Sequential

classifier = Sequential([
    Dense(64, activation='relu', input_shape=(encoding_dim,)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

classifier.fit(X_train_encoded, y_train,
               epochs=80,
               batch_size=32,
               validation_split=0.2,
               class_weight=class_weight_dict,
               callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
               verbose=0)

# Evaluación final
loss, acc = classifier.evaluate(X_test_encoded, y_test, verbose=0)
print(f"✅ Accuracy Autoencoder + Clasificador: {acc:.4f}")


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


✅ Accuracy Autoencoder + Clasificador: 0.3621
